In [1]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import os

In [2]:
def dictionary_and_embedding(self, input_file, size):
        self.size = size
        if not input_file:
            input_file = self.train_file_path
        #trees = pd.read_pickle(input_file)
        trees = self.train
        if not os.path.exists(self.root+'imtrain/embedding'):
            os.mkdir(self.root+'imtrain/embedding')
        from prepare_data import get_sequences

        def trans_to_sequences(ast):
            sequence = []
            get_sequences(ast, sequence)
            return sequence
        corpus = trees['code'].apply(trans_to_sequences)
        str_corpus = [' '.join(c) for c in corpus]
        trees['code'] = pd.Series(str_corpus)
        trees.to_csv(self.root+'imtrain/programs_ns.tsv')

        from gensim.models.word2vec import Word2Vec
        w2v = Word2Vec(corpus, size=size, workers=16, sg=1, min_count=3)
        w2v.save(self.root+'imtrain/embedding/node_w2v_' + str(size))

In [ ]:
def trans_to_sequences(ast):
            sequence = []
            get_sequences(ast, sequence)
            return sequence

In [113]:
all_data = pd.read_csv("./data/all_function_long_path_dropna.csv")

In [114]:
all_data

,Unnamed: 0,Unnamed: 0.1,id,bug,code,flaw,func_with_fix,vul_lines,vul_lines_fix,code_no_comment,long_path_combine,long_path_greedy,long_path,nodes,edge_list,flaw_loc,longest_path_token_num,path_num
0,0,0,0,1,pango_glyph_string_set_size (PangoGlyphString ...,CWE-189,pango_glyph_string_set_size (PangoGlyphString ...,\tstring->space = 1;\n\tstring->space *= 2;\n ...,\t{\n\t string->space = 4;\n\t}\n\t const gu...,pango_glyph_string_set_size (PangoGlyphString ...,"[[[[3, 2, 1, 2, 3], [0, 1, 2]], 0]]","[[[[3, 2, 1, 2, 3], [0, 1, 2]], 0]]","[[1, 2, 3]]","{1: [2, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [2, 3]]","[7, 8, 9, 11, 12]",5,1
1,1,1,1,1,"dispatch_cmd(conn c)\n{\n int r, i, timeout...",NaN,"dispatch_cmd(conn c)\n{\n int r, i, timeout...","return reply_msg(c, MSG_JOB_TOO_BI...",/* throw away the job body and res...,"dispatch_cmd(conn c)\n{\n int r, i, timeout...","[[[[5, 4, 3, 2, 1, 2, 3, 4, 6], [0, 1, 2, 3]],...","[[[[76, 75, 74, 73, 44, 42, 41, 37, 3, 2, 1, 2...","[[1, 2, 3, 4, 5], [1, 2, 3, 4, 6], [1, 2, 3, 4...","{1: [2, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [2, 3], [3, 4], [4, 5], [4, 6], [4, 7...",[40],19,39
2,2,2,2,1,static GIOChannel *irssi_ssl_get_iochannel(GIO...,CWE-20,static GIOChannel *irssi_ssl_get_iochannel(GIO...,NaN,static GIOChannel *irssi_ssl_get_iochannel(GIO...,static GIOChannel *irssi_ssl_get_iochannel(GIO...,"[[[[2, 1, 3], [0, 3, 4]], 0], [[[4, 1, 5], [0,...","[[[[2, 1, 3], [0, 3, 4]], 0], [[[4, 1, 5], [0,...","[[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7...","{1: [10, 'CursorKind.TRANSLATION_UNIT', 'test....","[[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7...",NaN,3,5
3,3,3,3,1,int irssi_ssl_handshake(GIOChannel *handle)\n{...,CWE-20,int irssi_ssl_handshake(GIOChannel *handle)\n{...,\tret = !chan->verify || irssi_ssl_verify(chan...,\tret = !chan->verify || irssi_ssl_verify(chan...,int irssi_ssl_handshake(GIOChannel *handle)\n{...,"[[[[3, 2, 1, 2, 4, 5, 6], [0, 1, 2, 3]], 0], [...","[[[[30, 29, 28, 23, 20, 19, 14, 4, 2, 1, 2, 4,...","[[1, 2, 3], [1, 2, 4, 5, 6], [1, 2, 4, 7, 8], ...","{1: [2, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [2, 3], [2, 4], [4, 5], [5, 6], [4, 7...",[36],18,10
4,5,5,5,1,"GIOChannel *net_connect_ip_ssl(IPADDR *ip, int...",CWE-20,"GIOChannel *net_connect_ip_ssl(IPADDR *ip, int...",\tssl_handle = irssi_ssl_get_iochannel(handle...,"GIOChannel *net_connect_ip_ssl(IPADDR *ip, int...","GIOChannel *net_connect_ip_ssl(IPADDR *ip, int...","[[[[2, 1, 3], [0, 4, 7]], 0], [[[4, 1, 4], [8,...","[[[[2, 1, 3], [0, 4, 7]], 0], [[[4, 1, 4], [8,...","[[1, 2], [1, 3], [1, 4]]","{1: [4, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [1, 3], [1, 4]]",[6],3,2
5,6,6,6,1,static void server_real_connect(SERVER_REC *se...,CWE-20,static void server_real_connect(SERVER_REC *se...,"\t\t\tnet_connect_ip_ssl(ip, port, own_ip, ser...","\t\t\tnet_connect_ip_ssl(ip, port, server->con...",static void server_real_connect(SERVER_REC *se...,"[[[[3, 2, 1, 2, 4], [0, 1]], 0], [[[5, 2, 1, 2...","[[[[8, 7, 6, 2, 1, 2, 6, 9, 10], [0, 1, 3, 4, ...","[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6, 7,...","{1: [2, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [2, 3], [2, 4], [2, 5], [2, 6], [6, 7...",[24],9,7
6,7,7,7,1,static int try_read_command(conn *c) {\n as...,CWE-20,static int try_read_command(conn *c) {\n as...,"if (strcmp(ptr, ""get "") && str...",if (ptr - c->rcurr > 100 ||\n ...,static int try_read_command(conn *c) {\n as...,"[[[[3, 2, 1, 2, 4, 5, 6], [0, 1, 6]], 0], [[[9...","[[[[70, 69, 68, 67, 66, 65, 64, 61, 59, 58, 53...","[[1, 2, 3], [1, 2, 4, 5, 6], [1, 2, 4, 5, 7, 8...","{1: [2, 'CursorKind.TRANSLATION_UNIT', 'test.c...","[[1, 2], [2, 3], [2, 4], [4, 5], [5, 6], [5, 7...",[101],28,20
7,8,8,8,1,test_js (void) {\n GString *result = g_stri...,CWE-264,test_js (void) {\n GString *result = g_stri...,/* uzbl commands can be run from javascrip...,NaN,test_js (void) {\n GString *result = g_stri...,"[[[[5, 4, 3, 2, 1, 2, 3, 4, 5], [0, 1, 2]], 0]]","[[[[5, 4, 3, 2, 1, 2

In [115]:
all_data["long_path_greedy_context"] = None
all_data["long_path_combine_context"] = None

In [116]:
corpus_long_path_combine_context = []
corpus_long_path_greedy_context = []
for index, row in all_data.iterrows():
    
    long_path_combine = eval(row["long_path_combine"])
    long_path_greedy = eval(row["long_path_greedy"])
    nodes = eval(row["nodes"])
    
    
    long_path_combine_all_context = []
    long_path_combine_each_context = []
    for path in long_path_combine:
        each_context = []
        len_path = len(path[0][0])
        i=0
        for node_id in path[0][0]:
            if i==0:
                long_path_combine_all_context.extend(nodes[node_id][2].split())
                each_context.append(nodes[node_id][2].split())
            elif i==len(path[0][0])-1:
                long_path_combine_all_context.extend(nodes[node_id][2].split())
                each_context.append(nodes[node_id][2].split())
            else:
                long_path_combine_all_context.append(nodes[node_id][1])
                each_context.append(nodes[node_id][1])
            i+=1
        long_path_combine_each_context.append([[each_context,path[0][1]],path[1]])
            
    long_path_greedy_all_context = []
    long_path_greedy_each_context = []
    for path in long_path_greedy:
        each_context = []
        len_path = len(path[0][0])
        i=0
        for node_id in path[0][0]:
            if i==0:
                long_path_greedy_all_context.extend(nodes[node_id][2].split())
                each_context.append(nodes[node_id][2].split())
            elif i==len(path[0][0])-1:
                long_path_greedy_all_context.extend(nodes[node_id][2].split())
                each_context.append(nodes[node_id][2].split())
            else:
                long_path_greedy_all_context.append(nodes[node_id][1])
                each_context.append(nodes[node_id][1])
            i+=1
        long_path_greedy_each_context.append([[each_context,path[0][1]],path[1]])
    
    all_data.loc[index,"long_path_greedy_context"] = str(long_path_greedy_each_context)
    all_data.loc[index,"long_path_combine_context"] = str(long_path_combine_each_context)
    
    corpus_long_path_combine_context.append(long_path_combine_all_context)
    corpus_long_path_greedy_context.append(long_path_greedy_all_context)
    if index%999 == 0:
        print(index)

0
999
1998
2997
3996
4995
5994
6993
7992
8991
9990
10989
11988
12987
13986
14985
15984
16983
17982
18981
19980
20979
21978
22977
23976
24975
25974
26973
27972
28971
29970
30969
31968
32967
33966
34965
35964
36963
37962
38961
39960
40959
41958
42957
43956
44955
45954
46953
47952
48951
49950
50949
51948
52947
53946
54945
55944
56943
57942
58941
59940
60939
61938
62937
63936
64935
65934
66933
67932
68931
69930
70929
71928
72927
73926
74925
75924
76923
77922
78921
79920
80919
81918
82917
83916
84915
85914
86913
87912
88911
89910
90909
91908
92907
93906
94905
95904
96903
97902
98901
99900
100899
101898
102897
103896
104895
105894
106893
107892
108891
109890
110889
111888
112887


In [117]:
all_data.to_csv("./data/all_function_long_path_add_context.csv")

In [118]:
print(all_data.loc[:,"longest_path_token_num"].max())

245


In [119]:
print(all_data.loc[:,"path_num"].max())

1607


In [80]:
print(all_data.loc[100]["long_path_greedy_context"])

[[[[['slots'], 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNARY_OPERATOR', 'CursorKind.IF_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.IF_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKind.IF_STMT', 'CursorKind.UNARY_OPERATOR', 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNEXPOSED_EXPR', ['npages']], [0, 1, 4, 137, 143, 83]], 0], [[[['npages'], 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNARY_OPERATOR', 'CursorKind.IF_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKind.IF_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.DECL_STMT', 'CursorKind.VAR_DECL', ['struct', 'kvm_memory_slot']], [0, 1, 4, 137, 138]], 0], [[[['slots'], 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNEXPOSED_EXPR', 'CursorKind.UNARY_OPERATOR', 'CursorK

In [66]:
print(all_data.loc[100]["long_path_greedy"])

[[[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82, 81, 80, 79, 75, 70, 10, 4, 1, 4, 10, 59, 60, 61, 62, 63], [0, 1, 4, 137, 143, 83]], 0], [[[83, 82

In [71]:
import numpy as np

In [30]:
print(all_data.loc[100]["long_path_greedy_context"])

[[['struct', 'kvm_memory_slot'], 'CursorKind.VAR_DECL', 'CursorKind.DECL_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKind.DECL_STMT', 'CursorKind.VAR_DECL', ['struct', 'kvm_memory_slot']], [['struct', 'kvm_memslots'], 'CursorKind.VAR_DECL', 'CursorKind.DECL_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKind.DECL_STMT', 'CursorKind.VAR_DECL', ['struct', 'kvm_memslots']], [['r'], 'CursorKind.DECL_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKind.DECL_STMT', ['base_gfn']], [['npages'], 'CursorKind.DECL_STMT', 'CursorKind.COMPOUND_STMT', 'CursorKind.FUNCTION_DECL', 'CursorKind.TRANSLATION_UNIT', 'CursorKind.FUNCTION_DECL', 'CursorKind.COMPOUND_STMT', 'CursorKi

In [127]:
corpus_long_path_combine_context[899]

['kiocb',
 'CursorKind.TRANSLATION_UNIT',
 'socket',
 'msghdr',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.PARM_DECL',
 'struct',
 'kiocb',
 'struct',
 'socket',
 'CursorKind.PARM_DECL',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.PARM_DECL',
 'struct',
 'msghdr',
 'size',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'flags',
 'sock',
 'CursorKind.DECL_STMT',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.DECL_STMT',
 'CursorKind.VAR_DECL',
 'struct',
 'sock',
 'CursorKind.VAR_DECL',
 'CursorKind.DECL_STMT',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.DECL_STMT',
 'target',
 'timeo',
 'CursorKind.DECL_STMT',
 'CursorKind.COMP

In [128]:
corpus_long_path_greedy_context[899]

['struct',
 'sock',
 'CursorKind.VAR_DECL',
 'CursorKind.DECL_STMT',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.DECL_STMT',
 'CursorKind.VAR_DECL',
 'sock',
 'CursorKind.DECL_STMT',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.DECL_STMT',
 'target',
 'timeo',
 'CursorKind.DECL_STMT',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.COMPOUND_STMT',
 'CursorKind.LABEL_STMT',
 'struct',
 'kiocb',
 'CursorKind.PARM_DECL',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.PARM_DECL',
 'struct',
 'socket',
 'struct',
 'msghdr',
 'CursorKind.PARM_DECL',
 'CursorKind.FUNCTION_DECL',
 'CursorKind.TRANSLATION_UNIT',
 'CursorKind.FUN

In [120]:
w2v_long_path_combine_context = Word2Vec(corpus_long_path_combine_context, size=128, workers=16, sg=1, min_count=1)
w2v_long_path_combine_context.save('./data/combine_context_node_w2v_' + str(128))

In [121]:
w2v_long_path_greedy_context = Word2Vec(corpus_long_path_greedy_context, size=128, workers=16, sg=1, min_count=1)
w2v_long_path_greedy_context.save('./data/greedy_context_node_w2v_' + str(128))

In [122]:
word2vec = Word2Vec.load('./data/greedy_context_node_w2v_128').wv
vocab = word2vec.vocab

In [123]:
vocab["struct"].index

7

In [124]:
word2vec2 = Word2Vec.load('./data/combine_context_node_w2v_128').wv
vocab2 = word2vec.vocab
vocab2["struct"].index

7

In [125]:
word2vec.syn0[7]

/data/jiahao/translate/py35venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([ 0.03962673, -0.5821681 ,  0.6806215 , -0.58382535,  0.37240928,
       -0.41011262, -0.25705555,  0.70524085,  0.39427438, -0.16272922,
       -0.5219827 , -0.0612922 , -0.7132256 ,  0.10779825,  0.18762761,
        0.93891895,  0.32277113,  0.33457646, -0.03689415, -0.5486174 ,
        0.45419142,  0.35563678, -1.3852633 , -0.1323439 ,  0.00943192,
        0.855719  ,  0.06036741, -0.07227991,  0.6500249 ,  0.16219226,
        0.08535507, -0.25982204,  0.24486968,  0.6954269 , -0.2721002 ,
       -0.0499494 , -0.4645691 , -0.38263968, -0.06367885,  0.49653718,
        0.09080105, -0.6956623 ,  0.58936787, -0.3410021 , -0.73592675,
       -0.4770473 , -0.31318435,  0.28172997, -0.01692084,  0.01596071,
       -0.4300737 , -0.22707906, -0.22344793,  0.1651187 , -0.08139113,
       -0.19606695,  0.13806756, -0.18877189, -0.46127963, -0.25800365,
       -0.35304883,  0.24081169, -0.5325076 , -0.66650873, -0.0515271 ,
        0.33542296, -0.22636043, -0.3345372 ,  0.06727792, -0.09

In [126]:
word2vec2.syn0[7]

/data/jiahao/translate/py35venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([ 0.4365097 ,  0.15085208,  0.41588485,  0.9200982 ,  0.23058663,
        0.58785945, -0.62066144,  0.7209357 , -0.3119585 , -0.17956956,
        0.16167904, -0.9113916 , -0.91213757, -0.51822674,  0.03990658,
        0.4918161 ,  0.5843434 , -0.19749734,  0.32847628, -0.68998766,
       -0.13466264, -0.8037226 , -0.2018529 , -0.19846784, -0.39283618,
        0.9160607 ,  0.83588386,  0.76853305, -0.45214382, -0.19158804,
        0.74581   , -0.12507445,  0.4666748 ,  1.0772203 , -0.18259735,
       -0.47450086,  0.21999255, -0.07829876,  0.00410272, -0.23324534,
        0.03134276, -0.19083919, -0.1882569 , -0.02763497,  0.19540356,
        0.17947038, -0.25534648,  0.08016355, -0.15298113,  0.49685755,
       -0.17574109, -0.2086786 ,  0.2799047 , -0.0623984 ,  0.0379011 ,
        0.28623787,  0.23768237, -0.50938   , -0.4591563 , -0.46622616,
        0.07398885, -0.6097083 , -0.732438  , -0.45113954, -0.09607723,
        0.7771706 , -0.49703643, -0.47775757, -0.28997034, -0.64

In [129]:
len(vocab)

64645

In [130]:
good_data = all_data.loc[(all_data["bug"]==0)]
bad_data = all_data.loc[(all_data["bug"]==1)]

In [131]:
len(good_data)

106555

In [132]:
len(bad_data)

6744

In [133]:
good_data = good_data.sample(n=len(good_data),random_state=123)
bad_data = bad_data.sample(n=len(bad_data),random_state=123)

In [134]:
train_good = good_data[:4046]
val_good = good_data[4046:4046+1349]
test_good = good_data[4046+1349:4046+1349+1349]
test_good_extra = good_data[4046+1349:4046+1349+1349*9]
train_bad = bad_data[:4046]
val_bad = bad_data[4046:4046+1349]
test_bad = bad_data[4046+1349:]

In [135]:
train = pd.concat([train_good,train_bad]).sample(frac=1, random_state=666)
val = pd.concat([val_good,val_bad]).sample(frac=1, random_state=666)
test = pd.concat([test_good,test_bad]).sample(frac=1, random_state=666)
im_test = pd.concat([test_bad,test_good_extra]).sample(frac=1, random_state=666)

In [136]:
train.to_csv("./data/train.csv")
val.to_csv("./data/val.csv")
test.to_csv("./data/test.csv")
im_test.to_csv("./data/im_test.csv")

In [40]:
all_data["flaw_loc"] = None

In [58]:
for index, row in all_data.iterrows():
    flaw_loc= []
    flaw_line_code = row["vul_lines"]
    if not str(flaw_line_code) == "nan":
        flaw_line_code = flaw_line_code.strip().split("\n")
        for each_flaw_line in range(len(flaw_line_code)):
            flaw_line_code[each_flaw_line] = flaw_line_code[each_flaw_line].strip()
        print(flaw_line_code)
        code = row["code"].strip().split("\n")
        for code_line in range(len(code)):
            code[code_line] = code[code_line].strip()
        print(code)
        for flaw_line in flaw_line_code:
            for line_loc , line in enumerate(code):
                if flaw_line == line:
                    flaw_loc.append(line_loc)
    print(flaw_loc)

['string->space = 1;', 'string->space *= 2;', 'if (string->space < 0)', 'g_warning ("glyph string length overflows maximum integer size, truncated");', 'new_len = string->space = G_MAXINT - 8;']
['pango_glyph_string_set_size (PangoGlyphString *string, gint new_len)', '{', 'g_return_if_fail (new_len >= 0);', '', 'while (new_len > string->space)', '{', 'if (string->space == 0)', 'string->space = 1;', 'string->space *= 2;', 'if (string->space < 0)', '{', 'g_warning ("glyph string length overflows maximum integer size, truncated");', 'new_len = string->space = G_MAXINT - 8;', '}', '', 'string->glyphs = g_realloc (string->glyphs, string->space * sizeof (PangoGlyphInfo));', 'string->log_clusters = g_realloc (string->log_clusters, string->space * sizeof (gint));', 'string->num_glyphs = new_len;', '}']
[7, 8, 9, 11, 12]
['return reply_msg(c, MSG_JOB_TOO_BIG);']
['dispatch_cmd(conn c)', '{', 'int r, i, timeout = -1;', 'size_t z;', 'unsigned int count;', 'job j;', 'unsigned char type;', 'char *s

[42, 52, 43, 44, 54, 20, 45, 55, 46, 49, 50, 42, 52, 53, 44, 54, 20, 45, 55, 56]
['struct pt_regs *regs, u64 addr)', 'do_perf_sw_event(PERF_TYPE_SOFTWARE, event_id, nr, nmi, &data, regs);']
['void __perf_sw_event(u32 event_id, u64 nr, int nmi,', 'struct pt_regs *regs, u64 addr)', 'struct perf_sample_data data;', 'int rctx;', '', 'preempt_disable_notrace();', 'rctx = perf_swevent_get_recursion_context();', 'if (rctx < 0)', 'return;', '', 'perf_sample_data_init(&data, addr);', '', 'do_perf_sw_event(PERF_TYPE_SOFTWARE, event_id, nr, nmi, &data, regs);', 'perf_swevent_put_recursion_context(rctx);', 'preempt_enable_notrace();', '}']
[1, 12]
['u64 nr, int nmi,', 'perf_swevent_event(event, nr, nmi, data, regs);']
['static void do_perf_sw_event(enum perf_type_id type, u32 event_id,', 'u64 nr, int nmi,', 'struct pt_regs *regs)', '{', 'struct swevent_htable *swhash = &__get_cpu_var(swevent_htable);', 'struct perf_event *event;', 'struct hlist_node *node;', 'struct hlist_head *head;', '', 'rcu_re

['r = copy_from_user(data, (void __user *)addr + offset, len);']
['int kvm_read_guest_page(struct kvm *kvm, gfn_t gfn, void *data, int offset,', 'int len)', '{', 'int r;', 'unsigned long addr;', '', 'addr = gfn_to_hva(kvm, gfn);', 'if (kvm_is_error_hva(addr))', 'return -EFAULT;', 'r = copy_from_user(data, (void __user *)addr + offset, len);', 'return -EFAULT;', 'return 0;', '}']
[9]
['memcpy(tp->fw_ver, &vpd_data[j], len);', 'strncat(tp->fw_ver, " bc ", vpdlen - len - 1);']
['static void tg3_read_vpd(struct tg3 *tp)', '{', 'u8 *vpd_data;', 'unsigned int block_end, rosize, len;', 'u32 vpdlen;', 'int j, i = 0;', '', 'vpd_data = (u8 *)tg3_vpd_readblock(tp, &vpdlen);', 'if (!vpd_data)', 'goto out_no_vpd;', '', 'i = pci_vpd_find_tag(vpd_data, 0, vpdlen, PCI_VPD_LRDT_RO_DATA);', 'if (i < 0)', 'goto out_not_found;', '', 'rosize = pci_vpd_lrdt_size(&vpd_data[i]);', 'block_end = i + PCI_VPD_LRDT_TAG_SIZE + rosize;', 'i += PCI_VPD_LRDT_TAG_SIZE;', '', 'if (block_end > vpdlen)', 'goto out_not_fou

['btrfs_file_extent_compression(leaf, fi) == 0 &&', 'u32 size = new_size - found_key.offset;', 'if (test_bit(BTRFS_ROOT_REF_COWS, &root->state))', 'inode_sub_bytes(inode, item_end + 1 -', 'new_size);', '* update the ram bytes to properly reflect', '* the new size of our item', 'btrfs_set_file_extent_ram_bytes(leaf, fi, size);', 'size =', 'btrfs_file_extent_calc_inline_size(size);', 'btrfs_truncate_item(root, path, size, 1);', 'inode_sub_bytes(inode, item_end + 1 -', 'found_key.offset);']
['int btrfs_truncate_inode_items(struct btrfs_trans_handle *trans,', 'struct btrfs_root *root,', 'struct inode *inode,', 'u64 new_size, u32 min_type)', '{', 'struct btrfs_path *path;', 'struct extent_buffer *leaf;', 'struct btrfs_file_extent_item *fi;', 'struct btrfs_key key;', 'struct btrfs_key found_key;', 'u64 extent_start = 0;', 'u64 extent_num_bytes = 0;', 'u64 extent_offset = 0;', 'u64 item_end = 0;', 'u64 last_size = new_size;', 'u32 found_type = (u8)-1;', 'int found_extent;', 'int del_item;', '

['int lxc_attach(const char* name, const char* lxcpath, lxc_attach_exec_t exec_function, void* exec_payload, lxc_attach_options_t* options, pid_t* attached_process)', '{', 'int ret, status;', 'pid_t init_pid, pid, attached_pid, expected;', 'struct lxc_proc_context_info *init_ctx;', 'char* cwd;', 'char* new_cwd;', 'int ipc_sockets[2];', '', 'if (!options)', 'options = &attach_static_default_options;', '', 'init_pid = lxc_cmd_get_init_pid(name, lxcpath);', 'if (init_pid < 0) {', 'ERROR("failed to get the init pid");', 'return -1;', '}', '', 'init_ctx = lxc_proc_get_context_info(init_pid);', 'if (!init_ctx) {', 'ERROR("failed to get context of the init process, pid = %ld", (long)init_pid);', 'return -1;', '}', '', 'personality = get_personality(name, lxcpath);', 'if (init_ctx->personality < 0) {', 'ERROR("Failed to get personality of the container");', 'lxc_proc_put_context_info(init_ctx);', 'return -1;', '}', 'init_ctx->personality = personality;', '', 'if (!fetch_seccomp(name, lxcpath, 

KeyboardInterrupt: 

In [50]:
print(all_data.iloc[1]["func_with_fix"])

dispatch_cmd(conn c)
{
    int r, i, timeout = -1;
    size_t z;
    unsigned int count;
    job j;
    unsigned char type;
    char *size_buf, *delay_buf, *ttr_buf, *pri_buf, *end_buf, *name;
    unsigned int pri, body_size;
    usec delay, ttr;
    uint64_t id;
    tube t = NULL;

    /* NUL-terminate this string so we can use strtol and friends */
    c->cmd[c->cmd_len - 2] = '\0';

    /* check for possible maliciousness */
    if (strlen(c->cmd) != c->cmd_len - 2) {
        return reply_msg(c, MSG_BAD_FORMAT);
    }

    type = which_cmd(c);
    dprintf("got %s command: \"%s\"\n", op_names[(int) type], c->cmd);

    switch (type) {
    case OP_PUT:
        r = read_pri(&pri, c->cmd + 4, &delay_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        r = read_delay(&delay, delay_buf, &ttr_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        r = read_ttr(&ttr, ttr_buf, &size_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        errno = 0;
       

In [52]:
print(all_data.iloc[1]["code"])

dispatch_cmd(conn c)
{
    int r, i, timeout = -1;
    size_t z;
    unsigned int count;
    job j;
    unsigned char type;
    char *size_buf, *delay_buf, *ttr_buf, *pri_buf, *end_buf, *name;
    unsigned int pri, body_size;
    usec delay, ttr;
    uint64_t id;
    tube t = NULL;

    /* NUL-terminate this string so we can use strtol and friends */
    c->cmd[c->cmd_len - 2] = '\0';

    /* check for possible maliciousness */
    if (strlen(c->cmd) != c->cmd_len - 2) {
        return reply_msg(c, MSG_BAD_FORMAT);
    }

    type = which_cmd(c);
    dprintf("got %s command: \"%s\"\n", op_names[(int) type], c->cmd);

    switch (type) {
    case OP_PUT:
        r = read_pri(&pri, c->cmd + 4, &delay_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        r = read_delay(&delay, delay_buf, &ttr_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        r = read_ttr(&ttr, ttr_buf, &size_buf);
        if (r) return reply_msg(c, MSG_BAD_FORMAT);

        errno = 0;
       

In [56]:
for i in range(3):
    print(i)

0
1
2
